In [ ]:
import os
import time

# --- 1. Sirf is line ko change karein ---
# 🔴 Pehli baar 'images1_archive.tar.gz' likhein
#    Doosri baar 'images2_archive.tar.gz' likhein, etc.
ARCHIVE_FILE_NAME = "Copy of images2_archive.tar.gz"

# --- 2. In paths ko hamesha same rehne dein ---
DRIVE_BASE_PATH = "/content/drive/MyDrive/images"
# Yeh hamesha same folder rahega taake images jama (collect) ho sakein
LOCAL_IMAGE_DIR = "/content/local_images"

# --- 3. Baqi code ab automatically kaam karega ---
DRIVE_TAR_PATH = os.path.join(DRIVE_BASE_PATH, ARCHIVE_FILE_NAME)
LOCAL_TAR_PATH = f"/content/{ARCHIVE_FILE_NAME}"

# Yeh line check karti hai ke folder hai ya nahi. Agar hai, to usay istemaal karti hai.
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
print(f"--- Setup Shuru ---")
print(f"Target Folder: {LOCAL_IMAGE_DIR}")

# --- FAST COPY ---
print(f"Copying {ARCHIVE_FILE_NAME}...")
start_time = time.time()
!cp "{DRIVE_TAR_PATH}" "{LOCAL_TAR_PATH}"
print(f"Copy complete in {(time.time() - start_time):.2f} seconds.")

# --- FAST UNPACK (ADD) ---
print(f"Extracting and ADDING images to {LOCAL_IMAGE_DIR}...")
start_time = time.time()
# Yeh command purani files ko delete nahi karta, sirf nayi files add karta hai
!tar -xzf "{LOCAL_TAR_PATH}" -C "{LOCAL_IMAGE_DIR}"
print(f"Extraction complete in {(time.time() - start_time):.2f} seconds.")

print(f"\n✅ --- READY TO TRAIN! Images are in {LOCAL_IMAGE_DIR} ---")

--- Setup Shuru ---
Target Folder: /content/local_images
Copying Copy of images2_archive.tar.gz...
Copy complete in 9.98 seconds.
Extracting and ADDING images to /content/local_images...
Extraction complete in 5.33 seconds.

✅ --- READY TO TRAIN! Images are in /content/local_images ---


In [ ]:
import os

# The path where you extracted the images
LOCAL_IMAGE_DIR = "/content/local_images"

try:
    file_count = len(os.listdir(LOCAL_IMAGE_DIR))
    print(f"✅ Success! Found {file_count} images in '{LOCAL_IMAGE_DIR}'.")
except FileNotFoundError:
    print(f"❌ Error: Could not find the directory '{LOCAL_IMAGE_DIR}'.")
except Exception as e:
    print(f"An error occurred: {e}")

✅ Success! Found 30051 images in '/content/local_images'.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

# --- 1. Configuration ---
# Colab GPU ke liye BATCH_SIZE barha dein
BATCH_SIZE = 128
IMG_WIDTH, IMG_HEIGHT = 224, 224

EPOCHS_HEAD = 10       # Stage 1 ke liye epochs
EPOCHS_FINE_TUNE = 20  # Stage 2 ke liye epochs

LEARNING_RATE_HEAD = 1e-3
LEARNING_RATE_FINE_TUNE = 1e-4 # Hum 1e-4 se shuru karein ge (ReduceLROnPlateau isay control karega)

# --- 2. Paths ---
# Yaqeen karein ke aapka setup cell chal chuka hai
IMAGE_DIR = '/content/local_images'
# Humari sab se clean CSV file
CLEAN_CSV_PATH = '/content/drive/MyDrive/images/master_tf_clean.csv'
MODEL_SAVE_PATH = '/content/drive/MyDrive/images/model1.keras'

# --- 3. Class Weights (Aapke original dataset se) ---
CLASS_WEIGHTS_DICT = {
    0: 1.3844487463126844, 1: 0.8326957196717676, 2: 0.33555644927050515,
    3: 2.2368930592791183, 4: 1.961664054336468, 5: 7.151666666666666
}
CLASS_NAMES = ['COVID-19', 'Lung Opacity', 'Normal', 'Pneumonia (Bacterial)', 'Pneumonia (Viral)', 'Tuberculosis']
NUM_CLASSES = len(CLASS_NAMES)

# --- 4. Load Full (Clean) Dataset ---
print(f"Loading clean CSV: {CLEAN_CSV_PATH}...")
try:
    df = pd.read_csv(CLEAN_CSV_PATH)
    # Filepath column banayein
    df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
    # Labels ko integers (0-5) mein convert karein
    class_indices = {name: i for i, name in enumerate(CLASS_NAMES)}
    df['label_idx'] = df['label'].map(class_indices)
except Exception as e:
    print(f"Error loading CSV or mapping paths: {e}")
    raise

print(f"Loaded {len(df)} clean image records.")

# --- 5. Split Data (Train, Validation, Test) ---
print("Splitting data into train, validation, and test sets...")
# 90% train+val, 10% test
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
# 80% train, 10% val
train_df, val_df = train_test_split(train_val_df, test_size=(len(test_df)/len(train_val_df)), random_state=42, stratify=train_val_df['label'])

print(f"Training images: {len(train_df)}")
print(f"Validation images: {len(val_df)}")
print(f"Test images: {len(test_df)}")

# --- 6. tf.data Pipeline (Corrupt images ke fix ke saath) ---
print("Building tf.data pipelines...")

def load_and_preprocess(filepath, label):
    img = tf.io.read_file(filepath)
    # decode_image har format (PNG, JPEG, etc.) ko handle karta hai
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    img = tf.keras.applications.densenet.preprocess_input(img) # DenseNet ki preprocessing
    return img, label

def build_dataset(df, augment=False):
    # Keras GPU-powered augmentation layers
    augmentation_layers = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ])

    dataset = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label_idx'].values))
    # AUTOTUNE Colab par behtar kaam karega (RAM ka masla nahi hoga)
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        dataset = dataset.map(lambda x, y: (augmentation_layers(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(BATCH_SIZE)
     # Memory mein cache karein (Colab par fast hai)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = build_dataset(train_df, augment=True)
val_dataset = build_dataset(val_df)
test_dataset = build_dataset(test_df) # Yeh aakhir mein istemaal hoga

# --- 7. Build Model Function ---
def build_densenet_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    base_model.trainable = False # Shuru mein base model ko freeze rakhein

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

print("Building DenseNet-121 model...")
model = build_densenet_model()

# --- 8. STAGE 1: Train the Head (Poora Code) ---
print("--- STAGE 1: Training the new head ---")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_HEAD),
    loss='sparse_categorical_crossentropy', # Kyunke labels integers (0-5) hain
    metrics=['accuracy']
)

# Callbacks (Stage 1 ke liye)
early_stopping_s1 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint_s1 = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True, verbose=1)

history_head = model.fit(
    train_dataset,
    epochs=EPOCHS_HEAD,
    validation_data=val_dataset,
    callbacks=[early_stopping_s1, model_checkpoint_s1],
    class_weight=CLASS_WEIGHTS_DICT # Class weights zaroor istemaal karein
)

print("Head training complete. Best weights (Stage 1) loaded.")
print(f"Best model abhi tak '{MODEL_SAVE_PATH}' mein save ho gaya hai.")

# --- 9. STAGE 2: Smart Fine-Tuning ---
print("--- STAGE 2: Smart Fine-tuning the top layers ---")

# (A) Sirf top layers ko unfreeze karein
base_model = model.layers[0] # DenseNet base model ko select karein
base_model.trainable = True

# Pehli 347 layers ko frozen rakhein (yeh basic shapes seekhti hain)
for layer in base_model.layers[:347]:
    layer.trainable = False

print(f"Total layers in base_model: {len(base_model.layers)}")
print("First 347 layers frozen. Only top layers will be fine-tuned.")

# (B) Model ko naye LR se compile karein
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINE_TUNE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# (C) Naye (Smart) Callbacks banayein
# Patience ko 10 kar dein
early_stopping_s2 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Nayi file save karein (taake Stage 1 ka model mehfooz rahe)
model_checkpoint_s2 = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True, verbose=1)
# Yeh naya callback hai
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)

print("Starting smart fine-tuning...")

# (D) Training run karein
history_fine_tune = model.fit(
    train_dataset,
    epochs=EPOCHS_FINE_TUNE,
    validation_data=val_dataset,
    callbacks=[early_stopping_s2, model_checkpoint_s2, reduce_lr], # Nayi list
    class_weight=CLASS_WEIGHTS_DICT
)

print("--- Smart Fine-Tuning Complete! ---")
print(f"Final best model '{MODEL_SAVE_PATH}' mein save ho gaya hai.")

# --- 10. Final Evaluation ---
print("\n--- Final Evaluation on Test Set ---")
print(f"Loading best model from '{MODEL_SAVE_PATH}'...")
# ModelCheckpoint ne jo sab se best model save kiya tha (chahe Stage 1 ya 2 se), usay load karein
model.load_weights(MODEL_SAVE_PATH)

print("Evaluating model on the hold-out test set...")
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

# --- IMPORTANT: Generate Classification Report ---
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

print("\nGenerating classification report and confusion matrix...")
# Test dataset se predictions aur labels collect karein
y_true = []
y_pred_probs = []

# tqdm progress bar
for images, labels in tqdm(test_dataset, desc="Generating Predictions"):
    y_true.extend(labels.numpy())
    y_pred_probs.extend(model.predict(images, verbose=0))

y_pred = np.argmax(y_pred_probs, axis=1)

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))

print("\n--- Classification Report (Test Set) ---")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

Loading clean CSV: /content/drive/MyDrive/images/master_tf_clean.csv...
Loaded 30036 clean image records.
Splitting data into train, validation, and test sets...
Training images: 24028
Validation images: 3004
Test images: 3004
Building tf.data pipelines...
Building DenseNet-121 model...
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
--- STAGE 1: Training the new head ---
Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4974 - loss: 1.2045
Epoch 1: val_loss improved from inf to 0.72666, saving model to /content/drive/MyDrive/images/model1.keras
188/188 ━━━━━━━━━━━━━━━━━━━━ 439s 2s/step - accuracy: 0.4979 - loss: 1.2030 - val_accuracy: 0.7367 - val_loss: 0.7267
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6934 - loss: 0.6960
Epoch 2: val_loss improved from 0.72666 to 0.65876, saving model to /content/drive/MyDrive/images/model1.keras
188/188 ━━━━━━━━━━━━━━━━━━━━ 365s 2s/step - accuracy: 0.6935 - loss: 0.6959 - val_accuracy: 0.7610 - val_loss: 0.658

AttributeError: 'InputLayer' object has no attribute 'layers'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

# --- 1. Configuration (Set Your Parameters) ---
BATCH_SIZE = 64
IMG_WIDTH, IMG_HEIGHT = 224, 224

EPOCHS_FINE_TUNE = 20
# Hum purana (original) low learning rate istemaal karein ge
LEARNING_RATE_FINE_TUNE = 1e-5

# --- 2. Paths (Apne hisaab se set karein) ---
SAVED_WEIGHTS_PATH = '/content/drive/MyDrive/images/model1.keras'
FINAL_MODEL_PATH = '/content/drive/MyDrive/images/v2_finetuned_best.keras'
IMAGE_DIR = '/content/local_images'
CLEAN_CSV_PATH = '/content/drive/MyDrive/images/master_tf_clean.csv'

# --- 3. Class Weights and Names ---
CLASS_WEIGHTS_DICT = {
    0: 1.38, 1: 0.83, 2: 0.33, 3: 2.23, 4: 1.96, 5: 7.15
}
CLASS_NAMES = ['COVID-19', 'Lung Opacity', 'Normal', 'Pneumonia (Bacterial)', 'Pneumonia (Viral)', 'Tuberculosis']
NUM_CLASSES = len(CLASS_NAMES)

# --- 4. Function Definitions (Simplified) ---

def load_and_preprocess(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    img = tf.keras.applications.densenet.preprocess_input(img)
    return img, label

def build_dataset(df, augment=False):
    augmentation_layers = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ])
    dataset = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label_idx'].values))
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if augment:
        dataset = dataset.map(lambda x, y: (augmentation_layers(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

def build_densenet_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# --- 5. Data Loading & Pipeline Execution ---
print(f"Loading data from {CLEAN_CSV_PATH}...")
df = pd.read_csv(CLEAN_CSV_PATH)
df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
class_indices = {name: i for i, name in enumerate(CLASS_NAMES)}
df['label_idx'] = df['label'].map(class_indices)
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
train_df, val_df = train_test_split(train_val_df, test_size=(len(test_df)/len(train_val_df)), random_state=42, stratify=train_val_df['label'])
train_dataset = build_dataset(train_df, augment=True)
val_dataset = build_dataset(val_df)
test_dataset = build_dataset(test_df)

# --- 6. Model Build and Weight Load (Skip Stage 1) ---
print("Building DenseNet-121 model structure...")
model = build_densenet_model()

print(f"--- STAGE 1 SKIPPED ---")
print(f"Loading best weights (83.42% Peak) from '{SAVED_WEIGHTS_PATH}'...")
try:
    model.load_weights(SAVED_WEIGHTS_PATH)
    print("✅ Weights loaded successfully. Starting Fine-Tuning from 83.42%.")
except Exception as e:
    print(f"❌ ERROR: Weights load nahi huin. {e}")
    raise

# --- 7. STAGE 2: Simple Fine-Tuning Execution (PURANA CODE) ---
print("\n--- STAGE 2: Simple Fine-tuning the FULL model ---")

# (A) Index 4 par Base model ko pakrein
base_model = model.layers[4]

# (B) Poora Base Model unfreeze karein
base_model.trainable = True

print("✅ Poora Base Model (Index 4) Fine-tuning ke liye unfreeze kar diya gaya hai.")

# (C) Model ko compile karein
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINE_TUNE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# (D) Callbacks (Puranay callbacks)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint(FINAL_MODEL_PATH, monitor='val_loss', save_best_only=True, verbose=1)

# NAYA CALLBACK: Agar 3 epochs tak behtari na ho to LR kam karo
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)

# (E) Training run karein
history_fine_tune = model.fit(
    train_dataset,
    epochs=EPOCHS_FINE_TUNE,
    validation_data=val_dataset,
    # Yahan list ko update karein
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    class_weight=CLASS_WEIGHTS_DICT
)
print("--- Simple Fine-Tuning Complete! ---")

Loading data from /content/drive/MyDrive/images/master_tf_clean.csv...
Building DenseNet-121 model structure...
--- STAGE 1 SKIPPED ---
Loading best weights (83.42% Peak) from '/content/drive/MyDrive/images/model1.keras'...
✅ Weights loaded successfully. Starting Fine-Tuning from 83.42%.

--- STAGE 2: Simple Fine-tuning the FULL model ---
✅ Poora Base Model (Index 4) Fine-tuning ke liye unfreeze kar diya gaya hai.
Epoch 1/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7945 - loss: 0.4745   
Epoch 1: val_loss improved from inf to 0.48736, saving model to /content/drive/MyDrive/images/v2_finetuned_best.keras
376/376 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - accuracy: 0.7945 - loss: 0.4745 - val_accuracy: 0.8202 - val_loss: 0.4874 - learning_rate: 1.0000e-05
Epoch 2/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 980ms/step - accuracy: 0.7956 - loss: 0.4722
Epoch 2: val_loss did not improve from 0.48736
376/376 ━━━━━━━━━━━━━━━━━━━━ 383s 1s/step - accuracy: 0.7956 - loss: 0.4722 - val_accuracy: 0.8

In [ ]:
# Zaroori imports
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import os
from tqdm.notebook import tqdm

# --- Configuration (Set Your Parameters) ---
# Yahan woh file honi chahiye jismein 81.89% ki best weights hain
FINAL_MODEL_PATH = '/content/drive/MyDrive/images/v2_finetuned_best.keras'
IMAGE_DIR = '/content/local_images'
CLEAN_CSV_PATH = '/content/drive/MyDrive/images/master_tf_clean.csv'
BATCH_SIZE = 64
IMG_WIDTH, IMG_HEIGHT = 224, 224

CLASS_NAMES = ['COVID-19', 'Lung Opacity', 'Normal', 'Pneumonia (Bacterial)', 'Pneumonia (Viral)', 'Tuberculosis']
NUM_CLASSES = len(CLASS_NAMES)

# --- 1. Data Loading & Pipeline Execution (Must match training setup) ---
print(f"Loading final model and data...")
df = pd.read_csv(CLEAN_CSV_PATH)
df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
class_indices = {name: i for i, name in enumerate(CLASS_NAMES)}
df['label_idx'] = df['label'].map(class_indices)
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
# Test dataset Final Evaluation ke liye
_, test_df = train_test_split(train_val_df, test_size=(len(test_df)/len(train_val_df)), random_state=42, stratify=train_val_df['label'])

# --- Build Dataset Function (Wohi Jo Training Mein Tha) ---
def load_and_preprocess(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    img = tf.keras.applications.densenet.preprocess_input(img)
    return img, label

def build_dataset(df):
    dataset = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label_idx'].values))
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

test_dataset = build_dataset(test_df)

# --- 2. Model Load and Evaluation ---
print(f"Loading best weights from '{FINAL_MODEL_PATH}'...")
try:
    # Model ko uske structure ke saath load karein
    model = tf.keras.models.load_model(FINAL_MODEL_PATH)
    print("✅ Model loaded successfully.")
except Exception as e:
    # Agar load_model fail ho to manual tareeqa istemaal karein
    print("❌ Model load error. Manual build and weight load ki zaroorat hai.")

    # Manual build (aapka purana function)
    def build_densenet_model():
        base_model = DenseNet121(weights=None, include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
        base_model.trainable = True # Fine-tuned model load kar rahe hain, isliye trainable=True rehne dein
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.5)(x)
        predictions = Dense(NUM_CLASSES, activation='softmax')(x)
        return Model(inputs=base_model.input, outputs=predictions)

    model = build_densenet_model()
    model.load_weights(FINAL_MODEL_PATH)

# --- 3. Final Prediction & Report ---
print("\nGenerating classification report and confusion matrix...")
# Test dataset se predictions aur labels collect karein
y_true = []
y_pred_probs = []

for images, labels in tqdm(test_dataset, desc="Generating Predictions"):
    y_true.extend(labels.numpy())
    y_pred_probs.extend(model.predict(images, verbose=0))

y_pred = np.argmax(y_pred_probs, axis=1)

print("\n--- Final Test Set Evaluation ---")
print(f"Total Test Samples: {len(y_true)}")

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))

print("\n--- Classification Report (V2 Fine-Tuned) ---")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

Loading final model and data...
Loading best weights from '/content/drive/MyDrive/images/v2_finetuned_best.keras'...
✅ Model loaded successfully.

Generating classification report and confusion matrix...


Generating Predictions:   0%|          | 0/47 [00:00<?, ?it/s]


--- Final Test Set Evaluation ---
Total Test Samples: 3004

--- Confusion Matrix ---
[[ 284   22   43    1    1   11]
 [  42  466   84    2    0    7]
 [  59   58 1308   41    5   21]
 [   0    0    1  215    8    0]
 [   0    0    8  126  121    0]
 [   2    0    2    0    0   66]]

--- Classification Report (V2 Fine-Tuned) ---
                       precision    recall  f1-score   support

             COVID-19       0.73      0.78      0.76       362
         Lung Opacity       0.85      0.78      0.81       601
               Normal       0.90      0.88      0.89      1492
Pneumonia (Bacterial)       0.56      0.96      0.71       224
    Pneumonia (Viral)       0.90      0.47      0.62       255
         Tuberculosis       0.63      0.94      0.75        70

             accuracy                           0.82      3004
            macro avg       0.76      0.80      0.76      3004
         weighted avg       0.84      0.82      0.82      3004

